## Anydice: two-colored polyhedral dice pool

[StackExchange question.](https://rpg.stackexchange.com/questions/198579/anydice-two-colored-polyhedral-dice-pool)

1. Roll two pools of dice, not necessarily all of the same size: a "green" pool and a "red" pool.
2. Green dice and red dice that rolled the same number eliminate each other 1:1.
3. If the two highest remaining dice are green, or there are only green dice remaining, it's a full success.
4. If the two highest remaining dice are red, or there are only green dice remaining, it's a failure.
5. Otherwise (the two highest dice are of different colors, or no dice remain), it's a partial success.

In [1]:
import piplite
await piplite.install("icepool")

import icepool
from icepool import d4, d6, d8, d10, d12, OutcomeCountEval

class GreenRed(OutcomeCountEval):
    def next_state(self, state, outcome, green, red):
        # State is the number of top-two dice for green and red.
        top_green, top_red = state or (0, 0)
        # If there are remaining places in the top two...
        remaining_top_two = 2 - (top_green + top_red)
        if remaining_top_two > 0:
            # Compute the number of non-eliminated dice that rolled this outcome.
            net = green - red
            # Then add them to the winning team's top two.
            if net > 0:
                top_green += min(net, remaining_top_two)
            elif net < 0:
                top_red += min(-net, remaining_top_two)
        return top_green, top_red
    
    def final_outcome(self, final_state, *pools):
        top_green, top_red = final_state
        if (top_green > 0) and not (top_red > 0):
            return 2
        elif (top_red > 0) and not (top_green > 0):
            return 0
        else:
            return 1
    
    def direction(self, *_):
        # See outcomes in descending order.
        return -1
    
green_red = GreenRed()
# The argument lists are implicitly cast to pools.
print(green_red.eval([d10, d8], [d6, d8]))

Denominator: 3840

| Outcome | Weight | Probability |
|--------:|-------:|------------:|
|       0 |    265 |   6.901042% |
|       1 |   2784 |  72.500000% |
|       2 |    791 |  20.598958% |



In [2]:
# A larger calculation.
print(green_red.eval([d12, d10, d8, d6, d4, d12, d10, d8, d6, d4], [d12, d10, d8, d6, d4, d12, d10, d8, d6, d4]))

Denominator: 281792804290560000

| Outcome |             Weight | Probability |
|--------:|-------------------:|------------:|
|       0 |  67701912081930556 |  24.025423% |
|       1 | 146388980126698888 |  51.949155% |
|       2 |  67701912081930556 |  24.025423% |

